# Bedrock AgentCore Workflow

This notebook provides a workflow for building, pushing, and deploying an AI agent to AWS Bedrock AgentCore.

## 📋 Table of Contents

| Step | Description | Key Actions |
|------|-------------|-------------|
| [Step 1](#step-1-install-dependencies-and-import-packages) | Install Dependencies and Import Packages | UV package installation, Python imports |
| [Step 2](#step-2-load-configuration) | Load Configuration | Parse aws creds and set deployment parameters |
| [Step 3](#step-3-initialize-aws-clients) | Initialize AWS Clients | Create boto3 clients for ECR and AgentCore |
| [Step 4](#step-4-create-ecr-repository) | Create ECR Repository | Check/create ECR repository for container images |
| [Step 5](#step-5-build-container-image) | Build Container Image | Use buildx for ARM64 platform |
| [Step 6](#step-6-authenticate-with-ecr) | Authenticate with ECR | AWS CLI ECR login for image push |
| [Step 7](#step-7-push-image-to-ecr) | Push Image to ECR | Upload container image to ECR repository |
| [Step 8](#step-8-deploy-to-bedrock-agentcore) | Deploy to Bedrock AgentCore | Create AgentCore runtime with container |
| [Step 9](#step-9-monitor-deployment-status) | Monitor Deployment Status | Track runtime deployment progress |
| [Step 10](#step-10-deployment-summary) | Deployment Summary | Final status, links, and next steps |

## 🚀 Quick Navigation

- **Setup Phase**: [Steps 1-3](#step-1-install-dependencies-and-import-packages) - Environment and AWS setup
- **Build Phase**: [Steps 4-5](#step-4-create-ecr-repository) - Container repository and image build
- **Deploy Phase**: [Steps 6-8](#step-6-authenticate-with-ecr) - Image push and AgentCore deployment
- **Monitor Phase**: [Steps 9-10](#step-9-monitor-deployment-status) - Status monitoring and summary

## Prerequisites

- **UV Package Manager**: MANDATORY - UV must be installed and available
- **AWS CLI**: Configured with appropriate permissions
- **Docker**: Installed and running with buildx support
- **Python 3.12+**: Required for Strands SDK compatibility
- **Access to AWS Bedrock AgentCore**: Service must be available in your region

## Required IAM Permissions

Your AWS credentials need the following permissions:
- `ecr:*` (for ECR operations)
- `bedrock-agentcore:*` (for AgentCore operations)
- `iam:PassRole` (for execution role)
- `logs:*` (for CloudWatch logs)

## Key Features

- **ARM64 Container Builds**: Uses Docker buildx for Bedrock AgentCore compatibility
- **Reliable ECR Authentication**: Uses AWS CLI for robust ECR login
- **Comprehensive Error Handling**: Clear error messages and troubleshooting steps
- **Progress Monitoring**: Real-time status updates throughout deployment

---

## Step 1: Install Dependencies and Import Packages

In [ ]:
# Install required packages using UV
!uv add boto3 docker python-dotenv strands-agents bedrock-agentcore pydantic

In [ ]:
import boto3
import docker
import json
import time
import base64
import yaml
from datetime import datetime
from pathlib import Path
import os
from dotenv import load_dotenv
import uuid

# Load environment variables
load_dotenv()

print("✅ Imported required packages")

## Step 2: Define Configuration Paramters

Seting up deployment parameters for use within the notebook.

In [ ]:
# Loading in our constants. 

print("📋 Loading configuration...")

# Get AWS account ID and region from current AWS session/profile
session = boto3.Session(profile_name='sso-swe-qa', region_name='eu-west-1' ) # The call (without any parameters) woult retrieve the AWS profile from the standard AWS credential chain
sts_client = session.client('sts')
AWS_ACCOUNT_ID = sts_client.get_caller_identity()['Account']
AWS_REGION = session.region_name

# Container registry parameters 
ECR_REPOSITORY_NAME = "bedrock_agentcore_notebook" # Defining our preferred container repo name
ECR_REPOSITORY = f"{AWS_ACCOUNT_ID}.dkr.ecr.{AWS_REGION}.amazonaws.com/{ECR_REPOSITORY_NAME}"
ECR_REGISTRY_URL = ECR_REPOSITORY.split('/')[0]  # Extract registry URL for login
IMAGE_TAG_LATEST = "latest"
FULL_IMAGE_URI = f"{ECR_REPOSITORY}:{IMAGE_TAG_LATEST}"

# Container build parameters
PLATFORM = "linux/arm64"
ENTRYPOINT = "src/main.py"

# AgentCore parameters
AGENT_NAME = "bedrock_agentcore_notebook"
NETWORK_MODE = "PUBLIC"
SERVER_MODE = "HTTP"

print("✅ Configuration loaded successfully")
print(f"   Account ID: {AWS_ACCOUNT_ID}")
print(f"   Region: {AWS_REGION}")
print(f"   ECR Repository: {ECR_REPOSITORY_NAME}")
print(f"   ECR Registry: {ECR_REGISTRY_URL}")
print(f"   Full Image URI: {FULL_IMAGE_URI}")
print(f"   Platform: {PLATFORM}")

## Step 3: Initialize AWS Clients

Create boto3 clients for ECR and Bedrock AgentCore services.

In [ ]:
# Initialize AWS clients
print("🔧 Initializing AWS clients...")

try:
    ecr_client = session.client('ecr')
    agentcore_client = session.client('bedrock-agentcore-control')
    sts_client = session.client('sts')
    iam_client = session.client('iam')

    # Verify AWS credentials
    identity = sts_client.get_caller_identity()
    print("✅ AWS clients initialized successfully")
    print(f"   Account: {identity['Account']}")
    print(f"   User/Role: {identity['Arn']}")
    print(f"   Region: {AWS_REGION}")
    
except Exception as e:
    print(f"❌ AWS client initialization failed: {e}")
    print("   Troubleshooting:")
    print("   1. Check AWS credentials: aws configure list")
    print("   2. Verify region: aws configure get region")
    print("   3. Test connectivity: aws sts get-caller-identity")
    raise

## Step 4: Create ECR Repository

Check if ECR repository exists and create it if needed.

In [ ]:
# Check if ECR repository exists, create if needed
print(f"📦 Managing ECR repository: {ECR_REPOSITORY_NAME}")

try:
    # Check if repository exists
    response = ecr_client.describe_repositories(repositoryNames=[ECR_REPOSITORY_NAME])
    repo = response['repositories'][0]
    print(f"✅ Repository exists: {repo['repositoryName']}")
    print(f"   Repository URI: {repo['repositoryUri']}")
    print(f"   Created: {repo['createdAt']}")
    
except ecr_client.exceptions.RepositoryNotFoundException:
    # Repository doesn't exist, check if we should create it
    print(f"✅ Repository does not exist yet. Creating: {ECR_REPOSITORY_NAME}")
    try:
        create_response = ecr_client.create_repository(
            repositoryName=ECR_REPOSITORY_NAME,
            imageScanningConfiguration={'scanOnPush': True},
            encryptionConfiguration={'encryptionType': 'AES256'}
        )
        repo = create_response['repository']
        print(f"✅ Repository created successfully: {repo['repositoryName']}")
        print(f"   Repository URI: {repo['repositoryUri']}")
        print("   Image scanning: Enabled")
        print("   Encryption: AES256")
    except Exception as create_error:
        print(f"❌ Repository creation failed: {create_error}")
        raise

## Step 5: Build Container Image

Build the container image for ARM64 platform using buildx.

In [ ]:
# Build ARM64 container image using Docker buildx
print(f"🔨 Building Docker image: {FULL_IMAGE_URI}")
print(f"   Platform: {PLATFORM}")
print(f"   Context: {Path.cwd()}")

try:
    # Initialize Docker client for verification
    docker_client = docker.from_env()
    
    # Use docker buildx for ARM64 builds (required for Bedrock AgentCore)
    print(f"🔧 Using Docker buildx for ARM64 build...")
    
    # Build using buildx with magic command - most reliable approach
    !docker buildx build --platform {PLATFORM} --tag {FULL_IMAGE_URI} --load .
    
    # Verify the image was created and get its info
    image = docker_client.images.get(FULL_IMAGE_URI)
    print(f"✅ Image built successfully: {image.id[:12]}")
    print(f"   Size: {image.attrs['Size'] / (1024*1024):.1f} MB")
    print(f"   Tags: {image.tags}")
    
    # Verify platform architecture
    image_info = docker_client.api.inspect_image(image.id)
    image_arch = image_info.get('Architecture', 'unknown')
    image_os = image_info.get('Os', 'unknown')
    print(f"   Architecture: {image_os}/{image_arch}")
    
    if image_arch == 'arm64' and image_os == 'linux':
        print(f"✅ Image built for correct platform: linux/arm64")
    else:
        print(f"⚠️  Image architecture: {image_os}/{image_arch} (expected: linux/arm64)")
    
except Exception as e:
    print(f"❌ Build failed: {e}")
    print("   Troubleshooting:")
    print("   1. Check Docker is running: docker version")
    print("   2. Verify buildx is available: docker buildx version")
    print("   3. Check Dockerfile exists: ls -la Dockerfile")
    print("   4. Verify build context: ls -la")
    raise

## Step 6: Authenticate with ECR

Authenticate with ECR using the Docker SDK.

In [ ]:
# Authenticate with ECR
print(f"🔐 Authenticating with ECR using Docker SDK...")
print(f"   Registry: {ECR_REGISTRY_URL}")
print(f"   Region: {AWS_REGION}")

try:
    # Get ECR authorization token using boto3
    auth_response = ecr_client.get_authorization_token()
    auth_data = auth_response['authorizationData'][0]
    
    # Decode the token to get username and password
    import base64
    token = base64.b64decode(auth_data['authorizationToken']).decode('utf-8')
    username, password = token.split(':')
    
    # Login using Docker SDK
    login_response = docker_client.login(
        username=username,
        password=password,
        registry=ECR_REGISTRY_URL,
        reauth=True
    )
    
    print("✅ ECR authentication successful via Docker SDK")
    print("   Method: Docker SDK login")
    print(f"   Registry: {ECR_REGISTRY_URL}")
    print(f"   Status: {login_response.get('Status', 'Success')}")
    
except Exception as e:
    print(f"❌ ECR authentication failed: {e}")
    print("   Troubleshooting:")
    print("   1. Check AWS credentials: aws sts get-caller-identity")
    print("   2. Verify ECR permissions: aws ecr describe-repositories")
    print("   3. Check Docker daemon is running: docker version")
    raise


## Step 7: Push Image to ECR

Checking that the image is available and then push to the ECR repository.

In [ ]:
# Verify image exists locally before pushing
print(f"🔍 Verifying that the local image is available...")

try:
    # Get the specific image
    image = docker_client.images.get(FULL_IMAGE_URI)
    print(f"✅ Found local image: {image.id[:12]}")
    print(f"   Tags: {image.tags}")
    print(f"   Size: {image.attrs['Size'] / (1024*1024):.1f} MB")
        
except docker.errors.ImageNotFound:
    print(f"❌ Image not found locally: {FULL_IMAGE_URI}")
    print(f"   Make sure the build step completed successfully")
    raise
except Exception as e:
    print(f"❌ Image verification failed: {e}")
    raise

In [ ]:
# Push the image to ECR
print(f"📤 Getting ready for pushing image to ECR: {FULL_IMAGE_URI}")

try:
    # Get the image object and push it
    image = docker_client.images.get(FULL_IMAGE_URI)
    repository_part = ECR_REPOSITORY  # Without :tag
    tag_part = "latest"
    
    print(f"📤 Now pushing image to ECR: {repository_part}:{tag_part}")
    
    # Push the image - using the full repository:tag URI
    push_logs = docker_client.images.push(
        repository=repository_part,
        tag=tag_part,
        stream=True,
        decode=True,
        auth_config={
            'username': username,
            'password': password
        }
    )
    
    # Print push progress
    for log in push_logs:
        print(f"Push log: {log}")  # See all logs
        if 'error' in log:
            raise Exception(f"Push error: {log['error']}")
        if 'status' in log:
            status = log['status']
            if 'progress' in log:
                print(f"   {status}: {log['progress']}")
            else:
                print(f"   {status}")
    
    print(f"✅ Image pushed successfully: {FULL_IMAGE_URI}")
    
    # Verify the push by listing images in ECR
    try:
        ecr_images = ecr_client.list_images(repositoryName=ECR_REPOSITORY_NAME)
        image_count = len(ecr_images['imageIds'])
        print(f"   ECR repository now contains {image_count} image(s)")
    except Exception as verify_error:
        print(f"   ⚠️  Could not verify ECR contents: {verify_error}")
    
except Exception as e:
    print(f"❌ Push failed: {e}")
    print("   Troubleshooting:")
    print(f"   1. Verify ECR authentication: docker login {ECR_REGISTRY_URL}")
    print(f"   2. Check repository exists: aws ecr describe-repositories --repository-names {ECR_REPOSITORY_NAME}")
    print(f"   3. Verify image exists locally: docker images {FULL_IMAGE_URI}")
    print(f"   4. Check ECR permissions: aws ecr get-repository-policy --repository-name {ECR_REPOSITORY_NAME}")
    raise

## Step 8: Deploy to Bedrock AgentCore

Create or update the Bedrock AgentCore runtime with the new container image.

In [ ]:
print("🔧 Setting up Bedrock AgentCore IAM service role and permissions")

# Creating a new service role
SERVICE_ROLE_NAME = f"AmazonBedrockAgentCoreRuntimeDefaultServiceRole_{AWS_ACCOUNT_ID}"

# The permission policy
restrictive_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:*",
                "bedrock-agentcore:*",
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents",
                "logs:DescribeLogGroups",
                "logs:DescribeLogStreams",
                "ecr:GetAuthorizationToken",
                "ecr:BatchCheckLayerAvailability",
                "ecr:GetDownloadUrlForLayer",
                "ecr:BatchGetImage"
            ],
            "Resource": "*"
        }
    ]
}

# Define the trust policy for bedrock-agentcore.amazonaws.com
trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock-agentcore.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

try:
    # Check if the service role exists already 
    response = iam_client.get_role(
        RoleName=SERVICE_ROLE_NAME
    )
    role = response['Role']
    role_arn = role['Arn']

    print(f"✅ Role exists: {SERVICE_ROLE_NAME}")
    print(f"   ARN: {role_arn}")
    print(f"   Created: {role['CreateDate']}")
    print(f"   Description: {role.get('Description', 'No description')}")

except Exception as e:
    # Role does not yet exist. Create the role. 
    print(f"✅ Creating role: {SERVICE_ROLE_NAME}")

    create_role_response = iam_client.create_role(
        RoleName=SERVICE_ROLE_NAME,
        AssumeRolePolicyDocument=json.dumps(trust_policy),
        Description="Service role for Amazon Bedrock AgentCore Runtime with minimal permissions"
    )

    role_arn = create_role_response['Role']['Arn']
    print(f"✅ Role created successfully: {role_arn}")

    # Add inline policy
    iam_client.put_role_policy(
        RoleName=SERVICE_ROLE_NAME,
        PolicyName="BedrockAgentCoreRuntimePolicy",
        PolicyDocument=json.dumps(restrictive_policy)
    )
    
    print("✅ Role created with restrictive permissions")
    


In [ ]:
# Create new agent runtime
AGENT_RUNTIME_NAME = f"agent_core_notebook_{AWS_ACCOUNT_ID}"
EXECUTION_ROLE_ARN = str(role_arn)

print(f"🔍 Checking existing agent runtimes...")

try:
    response = agentcore_client.list_agent_runtimes()
    
    # Check if our runtime exists already
    runtime_exists = False

    for runtime in response.get('agentRuntimes', []):
        if runtime['agentRuntimeName'] == AGENT_RUNTIME_NAME:
            print(f"✅ Agent runtime already exists!")
            print(f"   Runtime ARN: {runtime.get('agentRuntimeArn', 'N/A')}")
            print(f"   Status: {runtime.get('status', 'N/A')}")
            runtime_exists = True
            break

    if not runtime_exists:
        print(f"The agent runtime: {AGENT_RUNTIME_NAME} does not exit yet")
        print(f"🚀 Creating agent runtime: {AGENT_RUNTIME_NAME}")

        try:
            # Create agent runtime configuration
            agent_runtime_config = {
                'agentRuntimeName': AGENT_RUNTIME_NAME,
                'description': "GenAI AgentCore Runtime deployed from my notebook",
                'agentRuntimeArtifact': {
                    'containerConfiguration': {  
                        'containerUri': FULL_IMAGE_URI
                    }
                },
                'roleArn': EXECUTION_ROLE_ARN,
                'networkConfiguration': {
                    'networkMode': NETWORK_MODE
                    # Add networkModeConfig if using VPC mode
                },
                'protocolConfiguration': {
                    'serverProtocol': SERVER_MODE
                },
                'environmentVariables': {
                    'PYTHONPATH': '/app/src',
                    'DOCKER_CONTAINER': '1'
                }
            }

            response = agentcore_client.create_agent_runtime(**agent_runtime_config)
            agent_runtime_arn = response['agentRuntimeArn']
            agent_runtime_id = response['agentRuntimeId']

            print("✅ Agent runtime created successfully!")
            print(f"   Runtime ARN: {agent_runtime_arn}")
            print(f"   Runtime ID: {agent_runtime_id}")
            print(f"   Status: {response['status']}")
            print(f"   Version: {response['agentRuntimeVersion']}")
    
        except Exception as e:
            print(f"❌ Failed to create agent runtime: {e}")
            print("   Troubleshooting:")
            print("   1. Verify execution role exists: aws iam get-role --role-name {EXECUTION_ROLE_ARN.split('/')[-1]}")
            print("   2. Check Bedrock AgentCore permissions")
            print(f"   3. Verify image is accessible: aws ecr describe-images --repository-name {ECR_REPOSITORY_NAME}")
    
except Exception as e:
    print(f"❌ Failed to manage agent runtime: {e}")

## Step 9: Monitor Deployment Status

Monitor the agent runtime deployment status until it's ready.

In [ ]:
# Monitor deployment status
print(f"⏳ Monitoring deployment status...")
print(f"   Runtime ID: {agent_runtime_id}")

max_wait_time = 60  # 1 minute
check_interval = 5  # 5 seconds
elapsed_time = 0

while elapsed_time < max_wait_time:
    try:
        # Get current status
        response = agentcore_client.get_agent_runtime(agentRuntimeId=agent_runtime_id)
        status = response['status']
        
        print(f"   Status: {status} (elapsed: {elapsed_time}s)")
        
        if status == 'READY':
            print(f"✅ Agent runtime is now ACTIVE!")
            print(f"   Runtime ARN: {response['agentRuntimeArn']}")
            print(f"   Endpoint: {response.get('endpoint', 'Not available')}")
            
            # Store final runtime info for summary
            final_runtime_info = response
            break
            
        elif status in ['FAILED', 'STOPPED']:
            print(f"❌ Agent runtime deployment failed with status: {status}")
            if 'failureReason' in response:
                print(f"   Failure reason: {response['failureReason']}")
            break
        
        # Wait before next check
        time.sleep(check_interval)
        elapsed_time += check_interval
        
    except Exception as e:
        print(f"❌ Error checking status: {e}")
        break

if elapsed_time >= max_wait_time:
    print(f"⚠️  Deployment monitoring timed out after {max_wait_time}s")
    print(f"   Check the AWS console for current status")
    print(f"   Runtime may still be deploying in the background")

## Step 10: Deployment Summary

Display final deployment information and next steps.

In [ ]:
# Final deployment summary
print(f"\n🎉 Deployment Summary")
print(f"=" * 60)
print(f"Agent Runtime Name: {AGENT_RUNTIME_NAME}")
print(f"Agent Runtime ARN: {agent_runtime_arn}")
print(f"Agent Runtime ID: {agent_runtime_id}")
print(f"Container Image: {FULL_IMAGE_URI}")
print(f"Platform: {PLATFORM}")
print(f"Region: {AWS_REGION}")
print(f"Execution Role: {EXECUTION_ROLE_ARN}")

# Display final status if available
if 'final_runtime_info' in locals():
    print(f"Final Status: {final_runtime_info['status']}")
    if 'endpoint' in final_runtime_info:
        print(f"Endpoint: {final_runtime_info['endpoint']}")

print(f"\n📋 Next Steps:")
print(f"1. Test the agent runtime using the Bedrock AgentCore console")
print(f"2. Configure any additional settings or integrations")
print(f"3. Monitor logs in CloudWatch for any issues")
print(f"4. Update your application to use the new runtime ARN")
print(f"5. Set up monitoring and alerting for the runtime")

print(f"\n🔗 Useful Links:")
print(f"- ECR Repository: https://{AWS_REGION}.console.aws.amazon.com/ecr/repositories/private/{AWS_ACCOUNT_ID}/{ECR_REPOSITORY_NAME}")
print(f"- Bedrock AgentCore Console: https://{AWS_REGION}.console.aws.amazon.com/bedrock/home?region={AWS_REGION}#/agentcore")
print(f"- CloudWatch Logs: https://{AWS_REGION}.console.aws.amazon.com/cloudwatch/home?region={AWS_REGION}#logsV2:log-groups")
print(f"- IAM Role: https://console.aws.amazon.com/iam/home#/roles/{EXECUTION_ROLE_ARN.split('/')[-1]}")

print(f"\n🛠️  Troubleshooting Commands:")
print(f"# Check agent runtime status")
print(f"aws bedrock-agentcore get-agent-runtime --agent-runtime-id {agent_runtime_id} --region {AWS_REGION}")
print(f"")
print(f"# View CloudWatch logs")
print(f"aws logs describe-log-groups --log-group-name-prefix /aws/bedrock-agentcore --region {AWS_REGION}")
print(f"")
print(f"# List ECR images")
print(f"aws ecr list-images --repository-name {ECR_REPOSITORY_NAME} --region {AWS_REGION}")

print(f"\n✅ Deployment workflow completed!")